### Mount (Montar) Azure Data Lake para el Proyecto

In [0]:
def mount_adls(storage_account_name, container_name):
    # Obtener Secret Key de Key vault
    client_id = dbutils.secrets.get(scope="moviecjcc-secret-scope", key="service-principal-client-id")
    tenant_id = dbutils.secrets.get(scope="moviecjcc-secret-scope", key="service-principal-tenant-id")
    client_secret = dbutils.secrets.get(scope="moviecjcc-secret-scope", key="service-principal-client-secret")

    #Establecer configuraciones de Spark
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    #Desmontar (unmount) el montaje (mount) si ya existe
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
          dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
    
    #Mount (Montar) el contenedor del Storage Account
    dbutils.fs.mount(
    source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point = f"/mnt/{storage_account_name}/{container_name}",
    extra_configs = configs)

    #Listar los Mounts
    display(dbutils.fs.mounts())


### Montar (mount) el contenedor "bronze"

In [0]:
mount_adls("sacjccmoviehistory", "bronze")

### Montar (mount) el contenedor "silver"

In [0]:
mount_adls("sacjccmoviehistory", "silver")

### Montar (mount) el contenedor "gold"

In [0]:
mount_adls("sacjccmoviehistory", "gold") 

In [0]:
dbutils.fs.unmount("/mnt/sacjccmoviehistory/demo")

### Montar (mount) el contenedor "demo"

In [0]:
mount_adls("sacjccmoviehistory", "demo")